# Explanation of this Notebook

The goal of this notebook is to explore the dataset and have better comprehesion of it

## Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
file_path = "../data/raw/bank-additional-full.csv"
df = pd.read_csv(file_path, sep=';')

# Afficher les 5 premières lignes
df.head()

## DF_Info & DF_Describe

In [ ]:
# Aperçu général du dataset
df.info()

# Résumé statistique des données numériques
df.describe()

# Affichage des colonnes et de leurs types
print(df.columns)
